# Teams
### A team is a group of agents that work together to achieve a common goal.
### AgentChat supports several team presets:
### 1. RoundRobinGroupChat: A team that runs a group chat with participants taking turns in a round-robin fashion
### 2. SelectorGroupChat: A team that selects the next speaker using a ChatCompletion model after each message.
### 3. MagneticOneGroupChat: A generalist multi-agent system for solving open-ended web and file-based tasks across a variety of domains.
### 4. Swarm: A team that uses HandoffMessage to signal transitions between agents.

# Creating a team using RoundRobinGroupChat


In [3]:
import os
import asyncio
from autogen_agentchat.agents import AssistantAgent 
from autogen_agentchat.messages import StructuredMessage
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

In [7]:
# Creat an OpenAI model client
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", api_key=api_key)

In [8]:
# Create a primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant."
)

In [9]:
# Create a critic agent
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed."
)

In [10]:
# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the primary and critic agent.
team = RoundRobinGroupChat([primary_agent,critic_agent], termination_condition=text_termination)

# Running a team #

In [11]:
result = await team.run(task="Write a poem about the fall season.")
print(result.messages[-1].content)

This revised version is a wonderful transformation! You’ve effectively addressed the feedback and enhanced the poem’s depth and imagery. Here are some points where you've excelled and a couple of additional suggestions:

1. **Imagery and Detail**: The imagery is much richer and more varied now. Phrases like "a tapestry woven where days gently yawn" offer a fresh perspective on the fall season that is both visual and emotive.

2. **Emotional Reflection**: The addition of personal and shared experiences, such as gathering apples and sharing stories by the fire, creates a stronger emotional connection with readers. This aspect really shines now.

3. **Line Variation**: The varied line lengths add a nice dynamism to the poem and enhance the reading experience by creating a natural ebb and flow.

4. **Unique Expressions**: You’ve introduced some beautiful, unique phrases (e.g., "the crackle of firelight paints shadows that sway") that elevate the poem’s originality and magic.

Overall, this

# Observing a team \#

In [12]:
await team.reset() # Reset team for a new task.
async for message in team.run_stream(task="Write a poem about the fall season"):
    if isinstance(message, TaskResult):
        print("Stop reason: ", message.stop_reason)
    else:
        print(message)


id='dc6b9370-56c9-4895-bee3-6b6a78e16e8f' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 8, 2, 19, 18, 39, 348666, tzinfo=datetime.timezone.utc) content='Write a poem about the fall season' type='TextMessage'
id='e161a1cc-e38c-46be-a0bd-65bb7b9a5276' source='primary' models_usage=RequestUsage(prompt_tokens=26, completion_tokens=247) metadata={} created_at=datetime.datetime(2025, 8, 2, 19, 18, 46, 46868, tzinfo=datetime.timezone.utc) content="In the golden glow of autumn's grace,  \nLeaves descend in a waltzing embrace,  \nCrisp whispers fill the once summer air,  \nAs nature dons her seasonal flair.  \n\nTrees stand tall in hues of amber and rust,  \nTheir branches adorned in a flickering dust,  \nA tapestry woven with fire and cheer,  \nEach fluttering leaf tells the tale of the year.  \n\nPumpkins smile in their patches of gold,  \nHarvest moons rise, bright and bold,  \nFields once lush now wear a quilt,  \nOf earthy tones where memories are built.  \

In [13]:
await team.reset()
await Console(team.run_stream(task="Write a poem about the fall season.")) # Stream the message to the console

---------- TextMessage (user) ----------
Write a poem about the fall season.
---------- TextMessage (primary) ----------
In the hush of autumn's glow,  
Leaves like tapestries descend,  
Crimson, gold, and amber flow,  
Nature’s canvas, none can mend.  

Whispering winds through branches weave,  
A symphony of crisp delights,  
The harvest moon on nights we cleave,  
Drapes the world in silver lights.  

Pumpkin patches, fields aglow,  
Cider’s warmth in every mug,  
The scent of woodsmoke, soft and low,  
Embraces hearts like a snug rug.  

Sweaters pulled and fireside dreams,  
Hot cocoa swirls with warming cheer,  
The brook's laughter, soft as streams,  
Calls to those we hold so dear.  

As sun dips low and shadows grow,  
The earth prepares for winter’s fold,  
Yet in this dance of ebb and flow,  
Autumn’s stories still unfold.  

Each day a brush with fleeting time,  
Awakens thoughts of what has passed,  
In every note, a gentle rhyme,  
A season’s beauty, unsurpassed.  

-----

TaskResult(messages=[TextMessage(id='c6267e06-c64f-4851-b4a7-20537bc6a798', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 2, 19, 20, 37, 826618, tzinfo=datetime.timezone.utc), content='Write a poem about the fall season.', type='TextMessage'), TextMessage(id='506812b5-9ecd-4dde-a5bb-687dd2dbe11d', source='primary', models_usage=RequestUsage(prompt_tokens=27, completion_tokens=222), metadata={}, created_at=datetime.datetime(2025, 8, 2, 19, 20, 43, 401583, tzinfo=datetime.timezone.utc), content="In the hush of autumn's glow,  \nLeaves like tapestries descend,  \nCrimson, gold, and amber flow,  \nNature’s canvas, none can mend.  \n\nWhispering winds through branches weave,  \nA symphony of crisp delights,  \nThe harvest moon on nights we cleave,  \nDrapes the world in silver lights.  \n\nPumpkin patches, fields aglow,  \nCider’s warmth in every mug,  \nThe scent of woodsmoke, soft and low,  \nEmbraces hearts like a snug rug.  \n\nSweaters pulled and 

# Stopping a team \# 

In [15]:
# Create a new team with an external termination condition
external_termination = ExternalTermination()
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition = text_termination | external_termination)

In [20]:
# RUn a team in a background task.
run = asyncio.create_task(Console(team.run_stream(task="Write a short poem about a fall season.")))
# wait for some time
await asyncio.sleep(0.1)

# stop the team.
external_termination.set()

# Wait for the team to finish.
await run

---------- TextMessage (user) ----------
Write a short poem about a fall season.
---------- TextMessage (primary) ----------
Fading sun in a copper sky,  
Leaves like fire begin to fly.  
Chill in the air, a soft embrace,  
Autumn's beauty fills the space.  

Harvest gatherings, warm and bright,  
Cider's warmth on a cool, clear night.  
As nature dons her splendid gown,  
In the heart of fall, joy can be found.  


TaskResult(messages=[TextMessage(id='a253b6da-5ef7-491d-95f8-13879a145d88', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 2, 19, 32, 40, 145397, tzinfo=datetime.timezone.utc), content='Write a short poem about a fall season.', type='TextMessage'), TextMessage(id='9c5b00b8-abea-40ea-b304-687c3e41e903', source='primary', models_usage=RequestUsage(prompt_tokens=670, completion_tokens=77), metadata={}, created_at=datetime.datetime(2025, 8, 2, 19, 32, 41, 949783, tzinfo=datetime.timezone.utc), content="Fading sun in a copper sky,  \nLeaves like fire begin to fly.  \nChill in the air, a soft embrace,  \nAutumn's beauty fills the space.  \n\nHarvest gatherings, warm and bright,  \nCider's warmth on a cool, clear night.  \nAs nature dons her splendid gown,  \nIn the heart of fall, joy can be found.  ", type='TextMessage')], stop_reason='External termination requested')

# Resuming a team \#

In [22]:
await Console(team.run_stream())

---------- TextMessage (primary) ----------
Thank you for the thoughtful feedback! I appreciate your suggestions and will keep them in mind for revision. Here’s a revised version incorporating your insights:

---

Fading sun in a copper sky,  
Leaves whisper secrets as they fly.  
Crisp air crackles, a sweet refrain,  
As footsteps crunch on earth’s tapestry plain.  

Harvest gatherings, laughter rings,  
Cider's warmth, the joy it brings.  
Nostalgic dreams in the evening's glow,  
In autumn's embrace, our hearts overflow.  

Each sunset painted, a bittersweet sigh,  
A reminder that seasons must pass us by.  
In the heart of fall, where moments are found,  
Life’s fleeting beauty wraps us around.

---

Let me know your thoughts!
---------- TextMessage (critic) ----------
This revised version of your poem is lovely! You've effectively incorporated several of the suggestions, enhancing both imagery and emotional depth. Here are some points that stand out:

1. **Imagery**: The line “Lea

TaskResult(messages=[TextMessage(id='c2f20b0a-1bd4-4afe-a4e7-b5bcb069d281', source='primary', models_usage=RequestUsage(prompt_tokens=985, completion_tokens=156), metadata={}, created_at=datetime.datetime(2025, 8, 2, 19, 34, 41, 451986, tzinfo=datetime.timezone.utc), content="Thank you for the thoughtful feedback! I appreciate your suggestions and will keep them in mind for revision. Here’s a revised version incorporating your insights:\n\n---\n\nFading sun in a copper sky,  \nLeaves whisper secrets as they fly.  \nCrisp air crackles, a sweet refrain,  \nAs footsteps crunch on earth’s tapestry plain.  \n\nHarvest gatherings, laughter rings,  \nCider's warmth, the joy it brings.  \nNostalgic dreams in the evening's glow,  \nIn autumn's embrace, our hearts overflow.  \n\nEach sunset painted, a bittersweet sigh,  \nA reminder that seasons must pass us by.  \nIn the heart of fall, where moments are found,  \nLife’s fleeting beauty wraps us around.\n\n---\n\nLet me know your thoughts!", typ

# Cancellation Token \#
### You can abort a call to run() or run_stream() during execution by setting a CancellationToken passed to the cancellation_token parameter.

#### Different from stopping a team, aborting a team will immediately stop the team and raise a CancelledError exception.

In [23]:
# Create a cancellation token.
cancellation_token = CancellationToken()

# Use another coroutine to run the team.
run = asyncio.create_task(
    team.run(
        task="Translate the poem to Spanish.",
        cancellation_token = cancellation_token,
    )
)

# Cancel the run.
cancellation_token.cancel()

try:
    result =   await run
except asyncio.CancelledError:
    print("Task was cancelled.")

Task was cancelled.
